In [7]:
import xarray as xr
import pandas as pd

In [9]:
folder_path = "../data/prediction_data"
file_name = "global-lam-inference-20230101T06-20230115T12.nc"

In [10]:
global_ds = xr.open_dataset(f"{folder_path}/{file_name}")
global_ds

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Julie/Desktop/wfrt_msc-thesis/data/prediction_data/global-lam-inference-20230101T06-20230115T12.nc'

In [5]:
global_ds.initial_date

<xarray.DataArray 'initial_date' (initial_date: 6)> Size: 48B
array(['2023-01-01T06:00:00.000000000', '2023-01-01T12:00:00.000000000',
       '2023-01-01T18:00:00.000000000', '2023-01-02T00:00:00.000000000',
       '2023-01-02T06:00:00.000000000', '2023-01-02T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * initial_date  (initial_date) datetime64[ns] 48B 2023-01-01T06:00:00 ... 2...
Attributes:
    long_name:  initial_date

In [8]:
date_range = pd.date_range(start='2023-01-01', end='2023-02-01')
print(date_range)

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-14', '2023-01-15', '2023-01-16',
               '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20',
               '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24',
               '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-28',
               '2023-01-29', '2023-01-30', '2023-01-31', '2023-02-01'],
              dtype='datetime64[ns]', freq='D')
